In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import sys
import os
from utils import ply_to_dataframe, test_model, plot_point_cloud_xyz
import laspy
from plyfile import PlyData
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

In [2]:
# DATA
data = "data/RA_points.ply"
df = ply_to_dataframe(data)
# LOAD MODEL
booster = xgb.Booster()
booster.load_model("cat-model-hans.json")

In [3]:
# Mapeo de clases 
mapeo_clases = {
    1: 0,
    2: 1,
    5: 2,
    11: 3,
    13: 4,
    14: 5,
    15: 6,
    18: 7,
    19: 8
}

# Aplicar el mapeo a la columna 'class'
df['class'] = df['class'].map(mapeo_clases)
df["class"]
# seleccion de variables
d = df.iloc[:, :3] # x,y,z
t = df["class"] # class o target

In [4]:
print(f'dataframe de la clases:{t}')
print(f'dataframe de las coordenadas: {d}')

dataframe de la clases:0          1
1          1
2          1
3          1
4          6
          ..
6825060    4
6825061    4
6825062    4
6825063    4
6825064    4
Name: class, Length: 6825065, dtype: int64
dataframe de las coordenadas:                   x           y         z
0       -123.019356 -121.934380  4.985402
1       -122.912193 -121.954681  4.996438
2       -122.829506 -122.109718  4.836177
3       -122.943787 -122.038712  4.884180
4       -122.700966 -132.812653  2.789504
...             ...         ...       ...
6825060   14.005584   -5.761790  3.609277
6825061   14.742920  -12.828125  3.425181
6825062   14.661809  -11.937943  3.584976
6825063   14.603197  -11.863029  3.620526
6825064   13.751492  -12.420531  3.364453

[6825065 rows x 3 columns]


In [5]:
# Estandarizar las características
scaler = StandardScaler()
X_ = scaler.fit_transform(d)
dtest_r = xgb.DMatrix(X_)
# Realizar predicciones con el modelo


In [6]:
pred = booster.predict(dtest_r)

In [7]:
pred.shape

(6825065, 9)

In [8]:
pred

array([[9.5294816e-03, 9.8252594e-01, 3.1068005e-08, ..., 1.9211693e-06,
        9.5871865e-04, 5.7973422e-04],
       [6.5294961e-03, 9.8588318e-01, 2.1180456e-08, ..., 1.0262154e-06,
        6.7446620e-04, 4.1875883e-04],
       [5.6332952e-05, 9.6484828e-01, 1.1951069e-07, ..., 2.8800650e-04,
        2.0654956e-02, 1.4136707e-02],
       ...,
       [3.0961242e-02, 6.3093668e-01, 2.7319245e-04, ..., 4.5372257e-03,
        2.5199333e-01, 7.5326405e-02],
       [3.0961242e-02, 6.3093668e-01, 2.7319245e-04, ..., 4.5372257e-03,
        2.5199333e-01, 7.5326405e-02],
       [3.2461211e-02, 6.2375993e-01, 9.3302988e-05, ..., 4.7570416e-03,
        2.8559190e-01, 4.6630852e-02]], dtype=float32)

In [9]:
clases_predichas = []

# Obtener las clases predichas para cada fila del array de probabilidades
for fila in pred:
    clase_predicha = np.argmax(fila)  # Obtener la clase con la probabilidad más alta
    clase_mapeada = mapeo_clases.get(clase_predicha)  # Mapear la clase
    if clase_mapeada is not None:
        clases_predichas.append(clase_mapeada)

valores_unicos = list(set(clases_predichas))


In [10]:
valores_unicos[:7]

[0, 1, 2]